In [1]:
import numpy as np
from torch.utils.data import Dataset
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')
import os
from tqdm import tqdm
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

import torch
from datasets.gait import CasiaQueryDataset, OUMVLPDataset
from models.SpatialTransformerTemporalConv import SpatialTransformerTemporalConv, SpatioTemporalTransformer
from datasets.augmentation import *
from torchvision import transforms
from datasets.graph import Graph
from einops import rearrange
from einops.layers.torch import Rearrange
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import torch
import torch.nn as nn
from evaluate import _evaluate_casia_b, evaluate, _evaluate_OUMVLP

### CASIA-B

In [2]:
val_transform = transforms.Compose([
        SelectSequenceCenter(60),
        remove_conf(enable=True),
        normalize_width,
        ToTensor()])
dataset_valid = CasiaQueryDataset(
    '/home/epinyoan/git/GaitSelfFormer/v2_all/data/casiab_npy',
    id_range='75-124', #'75-124', #'63-124', #'25-124'
    duplicate_bgcl=False,
    transform=ThreeCenterSequenceTransform(
                transform=val_transform, sequence_length=60))
val_loader = torch.utils.data.DataLoader(
    dataset_valid,
    batch_size=256,
    num_workers=8,
    pin_memory=True,
)

CasiaQueryDataset: 5375


In [3]:
# checkpoint = torch.load('/home/epinyoan/git/GaitSelfFormer/v2_all/save/unify/ablation_study/2022-10-04-12-40-45_11_nolocal_32_64_128_256_lr6e-3/checkpoint/last.pth')
# # checkpoint = torch.load('/home/epinyoan/git/GaitSelfFormer/v2_all/save/unify/ablation_study/2022-10-12-16-13-56_28_M_nolocal_32_64_128_256_lr6e-3/checkpoint/last.pth')
# # checkpoint = torch.load('/home/epinyoan/git/GaitSelfFormer/v2_all/save/unify/ablation_study/2022-10-05-10-42-42_15_S_nolocal_32_64_128_256_lr6e-3/checkpoint/last.pth')

# model = SpatialTransformerTemporalConv(
#             num_frame=60, in_chans=2, spatial_embed_dim=32, out_dim=128, num_joints=17, kernel_frame=31)



checkpoint = torch.load('/home/epinyoan/git/GaitSelfFormer/v2_all/save/unify/ablation_study/2022-10-11-23-08-08_26_spTempTransformer_lr6e-3/checkpoint/last.pth')
model = SpatioTemporalTransformer(
            num_frame=60, in_chans=2, spatial_embed_dim=32, out_dim=128, num_joints=17)

model.load_state_dict(checkpoint, strict=True)
model.cuda()
''

''

In [6]:
accuracy_avg, sub_accuracies, df = evaluate(
                val_loader, model, _evaluate_casia_b, gpu=False)

Test: [0/21]	Time 1.629 (1.629)	
Test: [10/21]	Time 0.944 (0.998)	
Test: [20/21]	Time 0.937 (0.973)	
               0        18        36        54        72        90       108  \
NM#5-6  0.909091  0.912121  0.937000  0.919192  0.919388  0.927174  0.932990   
BG#1-2  0.825253  0.832323  0.857576  0.857143  0.842268  0.802151  0.788776   
CL#1-2  0.761000  0.803000  0.810000  0.781633  0.777083  0.765909  0.773958   

          126    144    162    180      mean  
NM#5-6  0.918  0.925  0.905  0.855  0.914541  
BG#1-2  0.826  0.822  0.786  0.713  0.813863  
CL#1-2  0.758  0.765  0.757  0.738  0.771871  


In [157]:
accuracy_avg

0.8833455242904034

In [7]:
df

,0,18,36,54,72,90,108,126,144,162,180,mean
NM#5-6,0.909091,0.912121,0.937000,0.919192,0.919388,0.927174,0.932990,0.918,0.925,0.905,0.855,0.914541
BG#1-2,0.825253,0.832323,0.857576,0.857143,0.842268,0.802151,0.788776,0.826,0.822,0.786,0.713,0.813863
CL#1-2,0.761000,0.803000,0.810000,0.781633,0.777083,0.765909,0.773958,0.758,0.765,0.757,0.738,0.771871


### REPORT...

In [158]:
large_df = df.copy()
(large_df*100).round(1)

,0,18,36,54,72,90,108,126,144,162,180,mean
NM#5-6,94.4,94.9,94.6,96.3,95.3,96.3,95.3,94.7,95.3,94.7,92.2,94.9
BG#1-2,83.5,85.6,88.1,89.7,85.2,87.4,84.0,84.7,84.6,87.0,81.4,85.6
CL#1-2,81.2,83.6,82.3,83.5,84.5,84.8,86.9,88.9,87.0,85.7,81.6,84.5


In [130]:
medium_df = df.copy()
(medium_df*100).round(1)

,0,18,36,54,72,90,108,126,144,162,180,mean
NM#5-6,90.5,92.0,93.9,93.7,93.9,92.7,93.8,92.9,93.5,91.9,90.2,92.6
BG#1-2,80.7,84.2,85.0,85.7,84.9,84.0,83.6,86.3,85.3,83.9,79.0,83.9
CL#1-2,74.3,78.1,77.6,77.1,78.9,75.6,77.7,78.9,81.7,81.8,77.7,78.1


In [153]:
small_df = df.copy()
(small_df*100).round(1)

,0,18,36,54,72,90,108,126,144,162,180,mean
NM#5-6,74.7,76.0,77.4,79.9,78.3,75.6,69.8,75.0,77.0,75.1,67.8,75.2
BG#1-2,60.1,61.8,63.5,64.3,62.7,60.7,52.5,60.4,61.1,61.9,51.1,60.0
CL#1-2,54.3,56.9,56.0,56.1,59.2,52.4,47.4,51.4,53.8,59.8,52.6,54.5


### OUMVLP

In [23]:
val_transform = transforms.Compose([
        SelectSequenceCenter(30),
        normalize_width,
        ToTensor()])
dataset_valid = OUMVLPDataset(
    '/home/epinyoan/dataset/OUMVLP/alphapose/npy_test',
    sequence_length=30,
    transform=ThreeCenterSequenceTransform(
                transform=val_transform, sequence_length=30))
val_loader = torch.utils.data.DataLoader(
    dataset_valid,
    batch_size=256,
    num_workers=8,
    pin_memory=True,
)

In [24]:
checkpoint = torch.load('/home/epinyoan/git/GaitSelfFormer/v2_all/save/unify/ablation_study/2022-10-07-13-19-28_21_OUMVLP_sp64_64_128_256_512_lr6e-3/checkpoint/last.pth')
model = SpatialTransformerTemporalConv(
            num_frame=30, in_chans=3, spatial_embed_dim=64, out_dim=128, num_joints=17, kernel_frame=16)
model.load_state_dict(checkpoint, strict=True)
model.cuda()
''

''

In [25]:
accuracy_avg, sub_accuracies, df = evaluate(
                val_loader, model, _evaluate_OUMVLP, gpu=True)

Test: [0/525]	Time 1.286 (1.286)	
Test: [10/525]	Time 0.471 (0.510)	
Test: [20/525]	Time 0.475 (0.493)	
Test: [30/525]	Time 0.478 (0.487)	
Test: [40/525]	Time 0.481 (0.485)	
Test: [50/525]	Time 0.478 (0.484)	
Test: [60/525]	Time 0.481 (0.484)	
Test: [70/525]	Time 0.485 (0.484)	
Test: [80/525]	Time 0.485 (0.484)	
Test: [90/525]	Time 0.486 (0.484)	
Test: [100/525]	Time 0.487 (0.484)	
Test: [110/525]	Time 0.489 (0.484)	
Test: [120/525]	Time 0.487 (0.485)	
Test: [130/525]	Time 0.487 (0.485)	
Test: [140/525]	Time 0.487 (0.485)	
Test: [150/525]	Time 0.486 (0.485)	
Test: [160/525]	Time 0.488 (0.485)	
Test: [170/525]	Time 0.489 (0.486)	
Test: [180/525]	Time 0.487 (0.486)	
Test: [190/525]	Time 0.487 (0.486)	
Test: [200/525]	Time 0.488 (0.486)	
Test: [210/525]	Time 0.491 (0.486)	
Test: [220/525]	Time 0.488 (0.486)	
Test: [230/525]	Time 0.487 (0.486)	
Test: [240/525]	Time 0.488 (0.486)	
Test: [250/525]	Time 0.487 (0.486)	
Test: [260/525]	Time 0.489 (0.486)	
Test: [270/525]	Time 0.491 (0.486)	
Tes

In [21]:
df

,0,15,30,45,60,75,90,180,195,210,225,240,255,270,mean
0,83.800003,78.300003,75.000000,72.599998,69.000000,66.199997,62.500000,67.300003,63.900002,65.199997,64.500000,63.599998,61.400002,59.799999,68.099998
15,75.099998,88.599998,83.599998,81.300003,77.900002,75.300003,71.400002,69.300003,73.900002,71.900002,70.599998,69.000000,67.699997,64.500000,74.300003
30,77.699997,89.599998,92.900002,91.599998,88.400002,86.699997,83.300003,77.000000,78.400002,82.300003,81.400002,79.099998,77.699997,76.000000,83.000000
45,73.000000,86.599998,91.300003,93.199997,90.500000,88.900002,86.500000,75.699997,77.000000,81.800003,83.900002,81.400002,80.000000,78.400002,83.400002
60,63.799999,77.099998,81.599998,83.699997,90.400002,84.300003,81.099998,68.599998,70.099998,73.199997,75.699997,82.000000,75.400002,73.199997,77.199997
75,65.099998,80.300003,86.599998,89.300003,91.500000,92.800003,91.500000,72.599998,73.300003,78.400002,80.800003,81.599998,83.800003,82.000000,82.099998
90,62.000000,75.300003,82.699997,86.300003,87.400002,90.900002,92.900002,71.699997,70.900002,76.800003,79.599998,80.199997,82.800003,82.900002,80.199997
180,61.000000,68.000000,69.699997,69.000000,67.900002,67.000000,65.500000,88.300003,76.800003,73.400002,69.000000,63.700001,62.000000,58.200001,68.500000
195,65.800003,79.300003,78.900002,78.400002,76.599998,75.699997,73.400002,83.900002,90.900002,84.599998,82.300003,78.800003,75.900002,70.699997,78.199997
210,71.599998,82.800003,87.400002,87.500000,85.400002,84.699997,82.900002,86.400002,90.199997,92.800003,91.300003,88.199997,85.599998,81.199997,85.599998
